In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
import grader

# The New York Social Graph

[New York Social Diary](http://www.newyorksocialdiary.com/) provides a
fascinating lens onto New York's socially well-to-do.  The data forms a natural social graph for New York's social elite.  Take a look at this page of a recent [run-of-the-mill holiday party](http://www.newyorksocialdiary.com/party-pictures/2014/holiday-dinners-and-doers).

Besides the brand-name celebrities, you will notice the photos have carefully annotated captions labeling those that appear in the photos.  We can think of this as implicitly implying a social graph: there is a connection between two individuals if they appear in a picture together.

For this project, we will assemble the social graph from photo captions for parties dated December 1, 2014, and before.  Using this graph, we can make guesses at the most popular socialites, the most influential people, and the most tightly coupled pairs.

We will attack the project in three phases:
1. Get a list of all the photo pages to be analyzed.
2. Parse all of the captions on a sample page.
3. Parse all of the captions on all pages, and assemble the graph.

## Phase One

The first step is to crawl the data.  We want photos from parties on or before December 1st, 2014.  Go to the [Party Pictures Archive](http://www.newyorksocialdiary.com/party-pictures) to see a list of (party) pages.  We want to get the url for each party page, along with its date.

Here are some packagest that you may find useful.  You are welcome to use others, if you prefer.

In [3]:
import requests
import dill
from bs4 import BeautifulSoup
from datetime import datetime

We recommend using Python [Requests](http://docs.python-requests.org/en/master/) to download the HTML pages, and [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) to process the HTML.  Let's start by getting the [first page](http://www.newyorksocialdiary.com/party-pictures).

In [6]:
url = "http://www.newyorksocialdiary.com/party-pictures"
page = requests.get(url, params={"limit": 1000, "offset": 0})

Now, we process the text of the page with BeautifulSoup.

In [7]:
soup = BeautifulSoup(page.text, "lxml")

This page has links to 50 party pages. Look at the structure of the page and determine how to isolate those links.  Your browser's developer tools (usually `Cmd`-`Option`-`I` on Mac, `Ctrl`-`Shift`-`I` on others) offer helpful tools to explore the structure of the HTML page.

Once you have found a patter, use BeautifulSoup's [select](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors) or [find_all](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find) methods to get those elements.

In [ ]:
links = ...

There should be 50 per page.

Let's take a look at that first link.  Figure out how to extract the URL of the link, as well as the date.  You probably want to use `datetime.strptime`.  See the [format codes for dates](https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior) for reference.

In [ ]:
link = links[0]
# Check that the title and date match what you see visually.

For purposes of code reuse, let's put that logic into a function.  It should take the link element and return the URL and date parsed from it.

In [4]:
def get_link_date(element):
    link_tuple=[]
    for link in element:
        link_url = link.select('a')[0]['href']
        link_date = link.select('span')[3].text
        link_tuple.append((link_url, link_date))
    return link_tuple

You may want to check that it works as you expected.

Once that's working, let's write another function to parse all of the links on a page.  Thinking ahead, we can make it take a Requests [Response](http://docs.python-requests.org/en/master/api/#requests.Response) object and do the BeautifulSoup parsing within it.

In [5]:
def get_links(response):
    soup = BeautifulSoup(response.text, "lxml")
    div = soup.find_all('div', attrs={'class': 'views-row'}) 
    return get_link_date(div) # A list of URL, date pairs

If we run this on the previous response, we should get 50 pairs.

In [8]:
assert len(get_links(page)) == 50

But we only want parties with dates on or before the first of December, 2014.  Let's write a function to filter our list of dates to those at or before a cutoff.  Using a keyword argument, we can put in a default cutoff, but allow us to test with others.

With the default cutoff, there should be no valid parties on the first page.  Adjust the cutoff date to check that it is actually working.

In [9]:
def filter_by_date(links, cutoff=datetime(2014, 12, 1)):
    # Return only the elements with date <= cutoff
    valid_list=[]
    for l,d in links:
        date = datetime.strptime(d,'%A, %B %d, %Y')
        if (date <= cutoff):
            valid_list.append((l,date))
    return valid_list

In [10]:
assert len(filter_by_date(get_links(page))) == 0

Now we should be ready to get all of the party URLs.  Click through a few of the index pages to determine how the URL changes.  Figure out a strategy to visit all of them.

HTTP requests are generally IO-bound.  This means that most of the time is spent waiting for the remote server to respond.  If you use `requests` directly, you can only wait on one response at a time.  [requests-futures](https://github.com/ross/requests-futures) lets you wait for multiple requests at a time.  You may wish to use this to speed up the downloading process.

In [11]:
from requests_futures.sessions import FuturesSession

link_list = []

session = FuturesSession(max_workers=5)
urls = ["http://www.newyorksocialdiary.com/party-pictures?page=" + str(i) for i in range(29)]
futures = [session.get(url) for url in urls]    

for future in futures:
    link_list.extend(filter_by_date(get_links(future.result())))

In the end, you should have 1193 parties.

In [12]:
assert len(link_list) == 1193

In [187]:
link_list[1]

('/party-pictures/2014/gala-guests', datetime.datetime(2014, 11, 24, 0, 0))

In case we need to restart the notebook, we should save this information to a file.  There are many ways you could do this; here's one using `dill`.

In [14]:
dill.dump(link_list, open('nysd-links.pkd', 'w'))

To restore the list, we can just load it from the file.  When the notebook is restarted, you can skip the code above and just run this command.

In [ ]:
link_list = dill.load(open('nysd-links.pkd', 'r'))

## Question 1: histogram

Get the number of party pages for the 95 months (that is, month-year pair) in the data.  Notice that while the party codes might be written as "FRIDAY, FEBRUARY 28, 2014", in this output, you would have to represent the month-year code as "Feb-2014".  This can all be done with `strfime` and the [format codes for dates](https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior).

Plot the histogram for yourself.  Do you see any trends?

In [20]:
def histogram():
    dic={}
    for l,d in link_list:
        monthyear = d.strftime('%b-%Y')
        if monthyear in dic:
            dic[monthyear] += 1
        else:
            dic[monthyear] = 1
        
    return dic.items()    

In [21]:
#def histogram():
#    return [("Dec-2014", 1)] * 95  # Replace with the correct list

#grader.score(question_name='graph__histogram', func=histogram)

Your score:  1.0


## Phase Two

In this phase, we we concentrate on getting the names out of captions for a given page.  We'll start with [the benefit cocktails and dinner](http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood) for [Lenox Hill Neighborhood House](http://www.lenoxhill.org/), a neighborhood organization for the East Side.

Take a look at that page.  Note that some of the text on the page is captions, but others are descriptions of the event.  Determine how to select only the captions.

In [ ]:
captions = ...

By our count, there are about 110.  But if you're off by a couple, you're probably okay.

In [47]:
assert abs(len(captions) - 110) < 5

Let's encapsulate this in a function.  As with the links pages, we want to avoid downloading a given page the next time we need to run the notebook.  While we could save the files by hand, as we did before, a checkpointing library like [ediblepickle](https://pypi.python.org/pypi/ediblepickle/1.1.3) can handle this for you.  (Note, though, that you may not want to enable this until you are sure that your function is working.)

You should also keep in mind that HTTP requests fail occasionally, for transient reasons.  You should plan how to detect and react to these failures.   The [retrying module](https://pypi.python.org/pypi/retrying) is one way to deal with this.

In [30]:
import requests
url = "http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood"
response = requests.get(url)

soup = BeautifulSoup(response.text, "lxml")
div = soup.find_all(['div', 'td'], attrs={'class': 'photocaption'}) 
captions = []
for e in div:
    captions.append(e.text)
captions

[u"Glenn Adamson, Simon Doonan, Victoire de Castellane, Craig Leavitt, Jerome Chazen, Andi Potamkin, Ralph Pucci, Kirsten Bailey, Edwin Hathaway, and Dennis Freedman at the Museum of Art and Design's annual MAD BALL. ",
 u' Randy Takian ',
 u' Kamie Lightburn and Christopher Spitzmiller ',
 u' Christopher Spitzmiller and Diana Quasha ',
 u' Mariam Azarm, Sana Sabbagh, and Lynette Dallas ',
 u' Christopher Spitzmiller, Sydney Shuman, and Matthew Bees',
 u' Christopher Spitzmiller and Tom Edelman ',
 u' Warren Scharf and Sydney Shuman ',
 u' Amory McAndrew and Sean McAndrew ',
 u' Sydney Shuman, Mario Buatta, and Helene Tilney ',
 u' Katherine DeConti and Elijah Duckworth-Schachter ',
 u' John Rosselli and Elizabeth Swartz ',
 u' Stephen Simcock, Lee Strock, and Thomas Hammer ',
 u' Searcy Dryden, Lesley Dryden, Richard Lightburn, and Michel Witmer ',
 u' Jennifer Cacioppo and Kevin Michael Barba ',
 u' Virginia Wilbanks and Lacary Sharpe ',
 u' Valentin Hernandez, Yaz Hernandez, Chele F

In [397]:
assert abs(len(captions) - 110) < 5

In [254]:
def get_names(captions):
    import re
    names_list = []
    for caps in captions:
        expression = re.compile('[A-Z]\w+\s[A-Z]\w+')
        names = re.findall(expression,caps)
        for n in names:
            names_list.append(n)
        
    return names_list

In [ ]:
#caps = caps.strip()
#if ';' in caps or len(caps)<3:
#    continue
#    print caps
#caps = caps.replace('and',',')
#caps = caps.replace('\n','')
#caps = caps.replace('\r','')
#caps = caps.split(',')'''


In [115]:
name_stub = '[A-Z][a-z\']{0,2}[A-Z]?[a-z.]+'

In [150]:
names = get_captions(captions)
len(names)
#names

212

In [488]:
test = ' Shelly Malkin and Nathalie Gerschel Kaplan \n\n'
expression = re.compile('[A-Z]\w+\s[A-Z]\w+')
re.findall(expression,test)

['Shelly BOB', 'Nathalie Gerschel']

In [ ]:
assert captions == get_captions("/party-pictures/2015/celebrating-the-neighborhood")

Now that we have some sample captions, let's start parsing names out of those captions.  There are many ways of going about this, and we leave the details up to you.  Some issues to consider:

  1. Some captions are not useful: they contain long narrative texts that explain the event.  Try to find some heuristic rules to separate captions that are a list of names from those that are not.  A few heuristics include:
    - look for sentences (which have verbs) and as opposed to lists of nouns. For example, [nltk does part of speech tagging](http://www.nltk.org/book/ch05.html) but it is a little slow. There may also be heuristics that accomplish the same thing.
    - Similarly, spaCy's [entity recognition](https://spacy.io/docs/usage/entity-recognition) couble be useful here.
    - Look for commonly repeated threads (e.g. you might end up picking up the photo credits or people such as "a friend").
    - Long captions are often not lists of people.  The cutoff is subjective, but for grading purposes, *set that cutoff at 250 characters*.
  2. You will want to separate the captions based on various forms of punctuation.  Try using `re.split`, which is more sophisticated than `string.split`. **Note**: The reference solution uses regex exclusively for name parsing.
  3. You might find a person named "ra Lebenthal".  There is no one by this name.  Can anyone spot what's happening here?
  4. This site is pretty formal and likes to say things like "Mayor Michael Bloomberg" after his election but "Michael Bloomberg" before his election.  Can you find other ('optional') titles that are being used?  They should probably be filtered out because they ultimately refer to the same person: "Michael Bloomberg."
  5. There is a special case you might find where couples are written as eg. "John and Mary Smith". You will need to write some extra logic to make sure this properly parses to two names: "John Smith" and "Mary Smith".
  6. When parsing names from captions, it can help to look at your output frequently and address the problems that you see coming up, iterating until you have a list that looks reasonable. This is the approach used in the reference solution. Because we can only asymptotically approach perfect identification and entity matching, we have to stop somewhere.
  
**Questions worth considering:**
  1. Who is Patrick McMullan and should he be included in the results? How would you address this?
  2. What else could you do to improve the quality of the graph's information?

In [153]:
def sample_names():
    new_captions = get_captions(captions)
    sorted_captions = sorted(list(set(new_captions)))
    return sorted_captions[:100]
    #return ["Caroline Dean"] * 100


In [146]:
#sample_names()[:100]

## Question 2: sample_names

Once you feel that your algorithm is working well on these captions, parse all of the captions and extract all the names mentioned.  Sort them alphabetically, by first name, and return the first hundred.

In [154]:
#def sample_names():
#    return ["Caroline Dean"] * 100

#grader.score(question_name='graph__sample_names', func=sample_names)

Your score:  0.9


Now, run this sort of test on a few other pages.  You will probably find that other pages have a slightly different HTML structure, as well as new captions that trip up your caption parser.  But don't worry if the parser isn't perfect -- just try to get the easy cases.

## Phase Three

Once you are satisfied that your caption scraper and parser are working, run this for all of the pages.  If you haven't implemented some caching of the captions, you probably want to do this first.

In [489]:
# import requests
# url = "http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood"
# response = requests.get(url)

# soup = BeautifulSoup(response.text, "lxml")
# div = soup.find_all(['div', 'td'], attrs={'class': 'photocaption'}) 
# captions = []
# for e in div:
#     captions.append(e.text)
# captions

In [ ]:
# Scraping all of the pages could take 10 minutes or so.

In [180]:
def get_captions_all(response):
    soup = BeautifulSoup(response.text, "lxml")
    div = soup.find_all(['div', 'tr', 'td'], attrs={'class': 'photocaption'}) 
    captions = []
    for e in div:
        captions.append(e.text)
    return captions

In [224]:
all_captions = []

session = FuturesSession(max_workers=5)
urls = ["http://www.newyorksocialdiary.com/"+str(l[0]) for l in link_list] 
#urls = ["http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood"]
len(urls)

1193

In [226]:
futures = [session.get(url) for url in urls]    

for future in futures:
    all_captions.extend(get_captions_all(future.result()))

In [227]:
len(all_captions)

125701

In [255]:
all_names = []
all_names = get_names(all_captions)

In [256]:
len(all_names)
#len(set(all_names))

224162

For the remaining analysis, we think of the problem in terms of a
[network](http://en.wikipedia.org/wiki/Computer_network) or a
[graph](https://en.wikipedia.org/wiki/Graph_%28discrete_mathematics%29).  Any time a pair of people appear in a photo together, that is considered a link.  What we have described is more appropriately called an (undirected)
[multigraph](http://en.wikipedia.org/wiki/Multigraph) with no self-loops but this has an obvious analog in terms of an undirected [weighted graph](http://en.wikipedia.org/wiki/Graph_%28mathematics%29#Weighted_graph).  In this problem, we will analyze the social graph of the new york social elite.  We recommend using python's [networkx](https://networkx.github.io/) library.

In [ ]:
sorted_names = sorted(all_names)

In [304]:
# junk = ['Junior','Benefit','Client','Clinical','Associate','President','Director','Board','Executive','Studio','AA','News','Dancers','II','Dancers','Lifetime','Trustee','Music','friend']

# all_name_new = [n for n in sorted_names
#        if not any(word in n for word in junk)]

In [308]:
#sorted(all_name_new)[:20]

All in all, you should end up with over 100,000 captions and more than 110,000 names, connected in about 200,000 pairs.

In [439]:
from collections import Counter

def degree_counter():
    degree_list = []
    for name,count in Counter(all_name_new).most_common(100):
        degree_list.append((name.encode("utf-8"),count))
    return degree_list

## Question 3: degree

The simplest question to ask is "who is the most popular"?  The easiest way to answer this question is to look at how many connections everyone has.  Return the top 100 people and their degree.  Remember that if an edge of the graph has weight 2, it counts for 2 in the degree.

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 189.92
    "std": 87.8053034454
    "min": 124.0
    "25%": 138.0
    "50%": 157.0
    "75%": 195.0
    "max": 666.0

In [411]:
def name_tuple_creator(names_list):
    import itertools

    comb = []
    for L in range(0, len(names_list)+1):
        for subset in itertools.combinations(names_list, 2):
            if subset not in comb:
                #print(subset)
                comb.append(subset)
                
    return comb

In [480]:
def get_together_names(captions):
    import re
    names_list = []
    #print captions
    for caps in captions:
        caps = caps.strip()
        caps = caps.replace('\n','')
        expression = re.compile('[A-Z]\w+\s[A-Z]\w+')
        names = re.findall(expression,caps)
        #new_names = []
        #for n in names:
        #    if len(n) > 2 and len(n) < 250: 
        #        print names
        #        new_names.append(names)
        if len(names) > 2 and len(names) < 250:
            print names
            names_list.append(name_tuple_creator(names))
        
    return names_list

In [694]:
def get_caption_names(captions):
    import re
    captions_names=[]
    
    for c in captions:
#         if len(re.findall('[a-zA-Z]', c))>0: 
#             print c
        if len(c) > 3:
            names = get_names(c)
            if len(names) > 0:
                captions_names.append(name_tuple_creator(names))
        
    return captions_names

In [717]:
def get_names(caption):
    import re
    str1 = caption.strip()
    str1 = str1.encode("utf-8")
    #str1 = str1.replace('\n',' ')
    str1 = str1.replace('with',',')
    str1 = str1.replace('and and','and')
    list1 = re.split(",|Mr.|Dr.|Miss|Mrs.|Jr.|M.D.|Ph.D.|PhD|CEO|Mayor|President|Honorees|Honoree|\n", str1)
    list2 = [e.strip() for e in list1 if len(e.strip())>2 and e.strip() != 'and']
    
    names_list=[]
    for e in list2:
        if 'and ' in e:
            l3= e.split('and')
            if (len(l3[0]) == 0 and len(l3[1]) == 0):
                continue
            #print "l3", l3
            friend_check = [True for f in l3 if "friend" in str(f)]
            #print friend_check
            if friend_check:
                #print "friend found: "
                #names_list.append(e.split('friend',1)[0])
                e = e.split('friend',1)[0]
                #print e
                continue
            if  len(l3) > 2:
                continue
            first = e.split('and ')[0].strip()
            second = e.split('and ')[1].strip()
#             print first
#             print second
            if len(first) == 0 and len(second) == 0:
                continue
            elif len(first) == 0 and len(second) != 0:
                if second[0].isupper(): names_list.append(second)
                #print second + "second"
            elif len(second) == 0 and len(first) != 0:
                if first[0].isupper(): names_list.append(first)
                #print first + "first"

    #       #for husband and wife
            elif len(re.findall(' ',first)) == 0:
                if (len(second.split())>1):
                    first = first + ' '+ second.split()[1]  
                if first[0].isupper() and second[0].isupper():
                    names_list.append(first)
                    names_list.append(second)
    #                 print "here"
#                 print first
#                 print second
            else:
                if first[0].isupper() and second[0].isupper():
                    names_list.append(first)
                    names_list.append(second)
    #     #not a person name
        elif len(re.findall(' ',e))>2:
            continue
        else:
            #print e
            names_list.append(e.strip())
    return names_list


In [657]:
import itertools  # itertools.combinations may be useful
import networkx as nx

In [718]:
cap_sample = ['Mariam Azarm, Sana Sabbagh\n  ','   u',' his wife and Mariam Azarm\n\n, Sana Sabbagh,and Lynette Dallas ',' Kamie Lightburn and Christopher Spitzmiller ']
#cap_sample = ['Chuck Grodin \nDiana Rosario, Ali Sussman, Sarah Boll, Jen Zaleski, Alysse Brennan, and Lindsay Macbeth and friends']
#cap_sample = 'and '
#cap_sample =  [' Dr. Amy Cunningham-Bussel, Ray Mirra, and Dr. Tyler Janovitz']
#cap_sample =  [' Melissa Errico, Todd Hollander, and Natalia Bulgari '] 
#cap_sample = [' Emdens and Falkenbergs and']
relation_names = get_caption_names(cap_sample)
relation_names

[[('Mariam Azarm', 'Sana Sabbagh')],
 [('Sana Sabbagh', 'Lynette Dallas')],
 [('Kamie Lightburn', 'Christopher Spitzmiller')]]

In [683]:
#all_relationship[:10]

In [719]:
all_relationship = get_caption_names(all_captions)
len(all_relationship)

94410

In [682]:
# all_relationship = get_together_names(all_captions)
# len(all_relationship)

In [490]:
#all_relationship[:10]

In [491]:
#all_captions[:10]

In [720]:
flat_list = [item for sublist in all_relationship for item in sublist]
len(flat_list)

207505

In [721]:
flat_list[:10]

[('Les Lieberman', 'Barri Lieberman'),
 ('Les Lieberman', 'Isabel Kallman'),
 ('Les Lieberman', 'Trish Iervolino'),
 ('Les Lieberman', 'Ron Iervolino'),
 ('Barri Lieberman', 'Isabel Kallman'),
 ('Barri Lieberman', 'Trish Iervolino'),
 ('Barri Lieberman', 'Ron Iervolino'),
 ('Isabel Kallman', 'Trish Iervolino'),
 ('Isabel Kallman', 'Ron Iervolino'),
 ('Trish Iervolino', 'Ron Iervolino')]

In [685]:
all_relationship[:1]

[[('Les Lieberman', 'Barri Lieberman'),
  ('Les Lieberman', 'Isabel Kallman'),
  ('Les Lieberman', 'Trish Iervolino'),
  ('Les Lieberman', 'Ron Iervolino'),
  ('Barri Lieberman', 'Isabel Kallman'),
  ('Barri Lieberman', 'Trish Iervolino'),
  ('Barri Lieberman', 'Ron Iervolino'),
  ('Isabel Kallman', 'Trish Iervolino'),
  ('Isabel Kallman', 'Ron Iervolino'),
  ('Trish Iervolino', 'Ron Iervolino')]]

In [722]:
def degree():

    G = nx.MultiGraph()
#     for row in flat_list:
#         #print row
    for first,second in flat_list:
        G.add_node(first)
        G.add_node(second)
        #print first,second
        G.add_edge(first,second)
    
    return sorted(G.degree().items(), key = lambda x: -x[1])[:100]
    #print nx.pagerank_scipy(G)

result = degree()    

In [723]:
result[:100]

[('Jean Shafiroff', 694),
 ('Gillian Miniter', 512),
 ('Mark Gilbertson', 469),
 ('Geoffrey Bradfield', 365),
 ('Somers Farkas', 313),
 ('Eleanora Kennedy', 295),
 ('Sharon Bush', 293),
 ('Debbie Bancroft', 288),
 ('Kamie Lightburn', 277),
 ('Andrew Saffir', 273),
 ('Alina Cho', 272),
 ('Michael Bloomberg', 265),
 ('Bonnie Comley', 263),
 ('Jamee Gregory', 253),
 ('Muffie Potter Aston', 247),
 ('Alexandra Lebenthal', 233),
 ('Allison Aston', 231),
 ('Lucia Hwong Gordon', 226),
 ('Mario Buatta', 223),
 ('nald', 221),
 ('Stewart Lane', 213),
 ('rmott', 203),
 ('Bettina Zilkha', 202),
 ('Barbara Tober', 197),
 ('Patrick McMullan', 191),
 ('Yaz Hernandez', 188),
 ('Sylvester Miniter', 187),
 ('Deborah Norville', 186),
 ('Ellen V. Futter', 185),
 ('Karen LeFrak', 181),
 ('Lydia Fenet', 181),
 ('Roric Tobin', 180),
 ('Liz Peek', 177),
 ('Daniel Benedict', 176),
 ('Audrey Gruss', 175),
 ('Liliana Cavendish', 173),
 ('Grace Meigher', 172),
 ('Amy Fine Collins', 172),
 ('Museum', 171),
 ('Diana

In [724]:
#import heapq  # Heaps are efficient structures for tracking the largest
              # elements in a collection.  Use introspection to find the
              # function you need.
#def degree():
 #   print [('Alec Baldwin', 144)] * 10


grader.score(question_name='graph__degree', func=degree)

Your score:  0.92


## Question 4: pagerank

A similar way to determine popularity is to look at their
[pagerank](http://en.wikipedia.org/wiki/PageRank).  Pagerank is used for web ranking and was originally
[patented](http://patft.uspto.gov/netacgi/nph-Parser?patentnumber=6285999) by Google and is essentially the stationary distribution of a [markov
chain](http://en.wikipedia.org/wiki/Markov_chain) implied by the social graph.

Use 0.85 as the damping parameter so that there is a 15% chance of jumping to another vertex at random.

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 0.0001841088
    "std": 0.0000758068
    "min": 0.0001238355
    "25%": 0.0001415028
    "50%": 0.0001616183
    "75%": 0.0001972663
    "max": 0.0006085816

In [727]:
def pagerank():
    G = nx.MultiGraph()
    for row in all_relationship:
        #print row
        for first,second in row:
            G.add_node(first)
            G.add_node(second)
            G.add_edge(first,second)
    
    #return sorted(G.degree().items(), key = lambda x: -x[1])[:100]
    return sorted(nx.pagerank_scipy(G).items(), key = lambda x: -x[1])[:100]


In [728]:
#def pagerank():
#    return [('Martha Stewart', 0.00019312108706213307)] * 100

grader.score(question_name='graph__pagerank', func=pagerank)

Your score:  0.93


## Question 5: best_friends

Another interesting question is who tend to co-occur with each other.  Give us the 100 edges with the highest weights.

Google these people and see what their connection is.  Can we use this to detect instances of infidelity?

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 25.84
    "std": 16.0395470855
    "min": 14.0
    "25%": 16.0
    "50%": 19.0
    "75%": 29.25
    "max": 109.0

In [736]:
len(flat_list)

207505

In [709]:
best_friends = Counter(flat_list).most_common()[:100]
#best_friends = sorted(Counter(flat_list))#, key = lambda x: -x[1])[:100]
best_friends[:10]

[(('Gillian Miniter', 'Sylvester Miniter'), 60),
 (('Stewart Lane', 'Bonnie Comley'), 59),
 (('Sylvester Miniter', 'Gillian Miniter'), 57),
 (('Jamee Gregory', 'Peter Gregory'), 50),
 (('Ashley', 'rmott'), 47),
 (('Andrew Saffir', 'Daniel Benedict'), 46),
 (('Jean Shafiroff', 'Martin Shafiroff'), 42),
 (('Geoffrey Bradfield', 'Roric Tobin'), 35),
 (('Barbara Tober', 'Donald Tober'), 34),
 (('Jonathan Farkas', 'Somers Farkas'), 32)]

In [732]:
G = nx.MultiGraph()
# for row in all_relationship:
#     #print row
for first,second in flat_list:
    G.add_node(first)
    G.add_node(second)
    G.add_edge(first,second)


In [733]:
import operator

def best_friends():
    dict_friends = {}
    for g in G.edges():
        if g in dict_friends:
            dict_friends[g] += 1
        elif g[::-1] in dict_friends:
            dict_friends[g] += 1
        else:
            dict_friends[g] = 1
    
    return sorted(dict_friends.items(),key=operator.itemgetter(1),reverse=True)[:100]

#best_friends()    

In [708]:
def best_friends_flatlist():
    dict_friends = {}
    for g in flat_list:
        if g in dict_friends:
            dict_friends[g] += 1
        elif g in dict_friends:
            dict_friends[(g)] += 1
        else:
            dict_friends[g] = 1
    
    return sorted(dict_friends.items(),key=operator.itemgetter(1),reverse=True)[:100]

best_friends()[:2]   

[(('Gillian Miniter', 'Sylvester Miniter'), 117),
 (('Bonnie Comley', 'Stewart Lane'), 82)]

In [734]:
#def best_friends():
#    return [(('Michael Kennedy', 'Eleanora Kennedy'), 41)] * 100

grader.score(question_name='graph__best_friends', func=best_friends)

Your score:  0.9


*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*